<a href="https://colab.research.google.com/github/tjdrms0930/bit/blob/main/20221028.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter15

In [ ]:
from tensorflow.keras import models, layers
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf

In [ ]:
df = pd.read_csv('data/house_train.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum().sort_values(ascending=False).head(20)

In [ ]:
df.isnull()

In [ ]:
df = pd.get_dummies(df)
df.head()

In [ ]:
df = df.fillna(df.mean())

In [ ]:
# 속성별 관련도 추출하기
df_corr = df.corr()
df_corr_sort = df_corr.sort_values('SalePrice', ascending=False)
df_corr_sort['SalePrice'].head(10)

In [ ]:
# 시각화 : 상관도 그래프
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF']
sns.pairplot(df[cols])
plt.show()

In [ ]:
cols_train = ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF']
X_train_pre = df[cols_train]

y = df['SalePrice'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train_pre, y, test_size=0.2)

In [ ]:
model = models.Sequential()
model.add(layers.Dense(10, input_dim=X_train.shape[1], activation='relu'))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(40, activation='relu'))
model.add(layers.Dense(1))
model.summary()

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam')
early_stopping = EarlyStopping(monitor='val_loss', patience=20)
modelpath = 'data/mode/house.hdf5'
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=0,
                              save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.25, epochs=2000, batch_size=32,
                   callbacks=[early_stopping, checkpointer])

In [ ]:
# 학습결과 시각화
real_prices = [] # 실제값 넣을 리스트
pred_prices = [] # 예측값 넣을 리스트
X_num = []

n_iter = 0
y_prediction = model.predict(X_test).flatten()
for i in range(30):
    real = y_test[i]
    prediction = y_prediction[i]
    print("실제가격: {:.2f}, 예상가격: {:.2f}".format(real, prediction))
    real_prices.append(real)
    pred_prices.append(prediction)
    n_iter += 1
    X_num.append(n_iter)

In [ ]:
plt.plot(X_num, pred_prices, label='predicted price')
plt.plot(X_num, real_prices, label='real price')
plt.legend()
plt.show()

# Chapter16

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import sys

In [ ]:
(X_train, y_train),(X_test, y_test) = mnist.load_data()
print('train image: %d' % (X_train.shape[0]))
print('test image: %d' % (X_test.shape[0]))

In [ ]:
print(X_train.shape)

In [ ]:
plt.imshow(X_train[0], cmap='Greys')
plt.show()

In [ ]:
for x in X_train[0]:
    for i in x:
        sys.stdout.write('%-4s' % i)
    sys.stdout.write('\n')

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28*28)
X_train = X_train.astype(float) / 255

X_test = X_test.reshape(X_test.shape[0], 28*28).astype(float) / 255

print(y_train[0])

In [ ]:
print(X_train[0])

In [ ]:
y_train = to_categorical(y_train) # import to_categorical 사용
y_test = to_categorical(y_test) # 원-핫 인코딩 문자열을 0과 1로만 이루어진 숫자 형태로 만들어주는 과정

print(y_train[0])

In [ ]:
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import os

In [ ]:
model = models.Sequential()
model.add(layers.Dense(512, input_dim=28*28, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',
             metrics=['accuracy'])

model_dir = 'data/model'
if not os.path.exists(model_dir):
    os.mkdir(model_path)
    
modelpath = 'data/model/mnist.hdf5'
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1,
                              save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.25, epochs=30, batch_size=200,
                   verbose=0, callbacks=[checkpointer, early_stopping])

In [ ]:
print('Test Accuracy: %.4f' % (model.evaluate(X_test, y_test)[1]))

In [ ]:
# 검증셋과 학습셋의 오차를 저장합니다. 
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

# 그래프로 표현해 봅니다.
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시해 보겠습니다.
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

# Chapter 16-2

In [ ]:
from tensorflow.keras import models, layers
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf

In [ ]:
(X_train, y_train),(X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype(float) / 255
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype(float) / 255
y_train = to_categorical(y_train) 
y_test = to_categorical(y_test) 

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3,3), input_shape=(28, 28, 1),
                       activation='relu'))
model.add(layers.Conv2D(64, kernel_size=(3,3), activation='relu'))